In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
import time
import scipy.stats

from tqdm import tqdm
from fair21 import SpeciesID, Category, Config, Species, RunMode, Scenario, ClimateResponse, RunConfig, FAIR
from fair21.defaults import species_config_from_default

# Gill Thornhill: methane temperature feedback on lifetime is -0.6 +/- 4.5 %/K
# TODO: specify warming

In [ ]:
df_temp = pd.read_csv('../data/forcing/AR6_GMST.csv')
gmst = np.zeros(351)
gmst[100:271] = df_temp['gmst'].values

In [ ]:
samples = 100

In [ ]:
species_ids = {
    'ch4': SpeciesID('CH4', Category.CH4),
    'co2': SpeciesID('CO2', Category.CO2, run_mode=RunMode.CONCENTRATION),
    'n2o': SpeciesID('N2O', Category.N2O, run_mode=RunMode.CONCENTRATION),
}

In [ ]:
emitted_species = ['CH4']
dummy_species = ['CO2', 'N2O']
species_to_include = emitted_species + dummy_species

In [ ]:
df_emis = pd.read_csv('../data/rcmip/rcmip-emissions-annual-means-v5-1-0.csv')
df_conc = pd.read_csv('../data/rcmip/rcmip-concentrations-annual-means-v5-1-0.csv')

In [ ]:
conc_in = df_conc.loc[
    (df_conc['Scenario']=='ssp245') & (df_conc['Variable'].str.endswith("CH4")) & 
    (df_conc['Region']=='World'), '1750':'2100'
].interpolate(axis=1).values.squeeze()

In [ ]:
scenarios_to_include = ['ssp245']
scenarios = []

for iscen, scenario in enumerate(scenarios_to_include):
    list_of_species = []
    for ispec, species in enumerate(emitted_species):
        species_rcmip_name = species.replace("-", "")
        emis_in = df_emis.loc[
            (df_emis['Scenario']==scenario) & (df_emis['Variable'].str.endswith("|"+species_rcmip_name)) & 
            (df_emis['Region']=='World'), '1750':'2100'
        ].interpolate(axis=1).values.squeeze()

        list_of_species.append(Species(species_ids[species.lower()], emissions=emis_in))
    for species in dummy_species:
        list_of_species.append(Species(species_ids[species.lower()], concentration=np.ones(351)*species_config_from_default(species).baseline_concentration))
        
    scenarios.append(Scenario(scenario, list_of_species))

In [ ]:
ch4_lifetime = np.ones(samples) * 8.25 #scipy.stats.uniform.rvs(7.25, 2, size=samples, random_state=31018316)
ch4_iirf_temperature = np.zeros(samples) #scipy.stats.uniform.rvs(-0.42, 0.35, size=samples, random_state=33372001)
ch4_iirf_airborne = np.ones(samples) * 0.00032 #scipy.stats.uniform.rvs(0, 0.0005, size=samples, random_state=371210)
ch4_baseline_concentration = scipy.stats.norm.rvs(729.2, 9.4/1.645, size=samples, random_state=121270)

In [ ]:
configs = []

seedgen = 1876963
for imodel in tqdm(range(samples)):
    config_name = f"{imodel}"
    climate_response = ClimateResponse()
    seedgen = seedgen+399
    species_config = [species_config_from_default(species) for species in species_to_include]
    # CH4 is index 0
    #species_config[0].iirf_0 = None
    species_config[0].iirf_airborne = ch4_iirf_airborne[imodel]
    species_config[0].iirf_temperature = ch4_iirf_temperature[imodel]
    species_config[0].lifetime = ch4_lifetime[imodel]
    species_config[0].baseline_concentration = ch4_baseline_concentration[imodel]
    species_config[0].calculate_iirf0()  # needed if editing the other params
    species_config[1].species_id = species_ids['co2']
    species_config[2].species_id = species_ids['n2o']
    configs.append(Config(config_name, climate_response, species_config))

In [ ]:
configs[1].species_configs[0]

In [ ]:
start = time.time()
fair = FAIR(scenarios, configs, time=np.arange(1750.5, 2101))
fair.prescribe_temperature(gmst)
fair.run()
end = time.time()
print (f"{len(scenarios) * len(configs)} ensemble members in {end - start}s.")

In [ ]:
pl.plot(fair.concentration_array[:,0,:,0,0]);
pl.plot(conc_in, color='k')

In [ ]:
pl.plot(fair.alpha_lifetime_array[:,0,:,0,0]);

In [ ]:
pl.plot(fair.alpha_lifetime_array[:,0,:,0,0] * fair.lifetime_array[:,0,:,0,0]);